In [1]:
import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1);
import numpy as np

In [2]:
import re
import os

In [3]:
def parsing_tweets(file_name):
    '''
    input: archivo con los tweets brutos
    output: archivo csv con los tweets limpios con 3 features:
    - fecha
    - texto limpio de @..., hashtags, urls y pics. Signo $ reemplazado por _
    - url (si hay)
    '''
    
    # archivo de lectura
    dir_name   = 'data/original'
    filename = os.path.join(dir_name, file_name)
    
    # regex
    match_date = re.compile(r"(\d{4}-\d{2}-\d{2})", re.IGNORECASE)
    match_url = re.compile(r"((https?|http):\/\/(www\.)?[a-z0-9\.:].*?(?=\s))", re.IGNORECASE)
    match_pic = re.compile(r"pic.twitter.com/[\w].*", re.IGNORECASE)
    match_arrobas = re.compile(r"(@\w*)", re.IGNORECASE)
    match_hashtags = re.compile(r"(#\w*)", re.IGNORECASE)
    match_signs = re.compile(r'(&(#\d*)+;)', re.IGNORECASE)  
   
    with open(filename,'r') as f:

        # inicializo dataframe
        tweets = pd.DataFrame()

        # lectura del file linea por linea (en caso de que el archivo sea muy grande)
        for line in f:

            # split de lineas para extraer fecha (más rápido que otro regex ??)
            l = line.split()

            if len(l)>0:
                # extraigo la url y la fecha, y '' si retorna None
                url = getattr(re.search(match_url, line), 'groups', lambda:[u""])()[0]
                date = getattr(re.search(match_date, line), 'groups', lambda:[u""])()[0]
                # uno los campos de texto
                text = ' '.join(l[5:])
                # reemplazo signo $ por _ y elimino la url del texto
                text = text.replace('$', '_')
                text = text.replace(url, '')
                # aplico las regex
                text = re.sub(match_pic, '', text)
                text = re.sub(match_signs, '', text)
                text = re.sub(match_arrobas, '', text)
                text = re.sub(match_hashtags, '', text)

                if date != '':
                    # creo diccionario para agregar a dataframe
                    d = {
                        'date' : date,
                        'text' : text,
                        'url'  : url
                    }
                    tweets = tweets.append(d, ignore_index=True)
    
    # retorno el dataframe
    return tweets

In [4]:
tweets_train = parsing_tweets('../data/nflx_twitter_train.txt')
tweets_test  = parsing_tweets('../data/nflx_twitter_test.txt')

In [5]:
tweets_train.to_csv('../data/tweets_train.csv')
tweets_test.to_csv('../data/tweets_test.csv')